In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    


In [15]:
#a+b = c, a + b = 5
a = Value(1)
b = Value(2)
c = Value(.34)
ytruth = Value(5)

w1 = Value(.56)
w2 = Value(-.34)
w3 = Value(.34)

In [ ]:
#a + b + c = d 
#d = 6

In [16]:

for i in range(10):
    ypred = a*w1 + b*w2 + c*w3
    loss = (ytruth - ypred)**2
    loss.backward()

    learning_rate = .05
    w1.data += learning_rate * -w1.grad
    w2.data += learning_rate * -w2.grad

    print(ypred)


Value(data=-0.004399999999999973, grad=-10.0088)
Value(data=2.4978000000000007, grad=-5.004399999999999)
Value(data=6.2511, grad=2.5022)
Value(data=9.378850000000002, grad=8.757700000000003)
Value(data=10.317175, grad=10.634350000000001)
Value(data=8.596912499999998, grad=7.193824999999997)
Value(data=5.078193749999998, grad=0.15638749999999568)
Value(data=1.5203781249999981, grad=-6.959243750000004)
Value(data=-0.2976265625000002, grad=-10.595253125000001)
Value(data=0.5331820312500017, grad=-8.933635937499997)
